In [1]:


lakehouse_silver_table=spark.conf.get('spark.executorEnv.lakehouse_silverTable')


print(lakehouse_silver_table)

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 3, Finished, Available, Finished)

abfss://Fabric_E2E@onelake.dfs.fabric.microsoft.com/Lakehouse_Silver_.Lakehouse/Tables


In [2]:
procedures_df = spark.read.format("csv").option("header","true").load("Files/raw/procedures.csv")
# df now is a Spark DataFrame containing CSV data from "Files/raw/procedures.csv".
display(procedures_df)

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4569aea3-41b8-402c-a737-625b419cd2c0)

In [3]:
pro_df=procedures_df.dropDuplicates()

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 5, Finished, Available, Finished)

In [4]:
display(pro_df)

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 777c5aae-771e-41c8-8258-820952798b59)

In [5]:
pro_cleaned=pro_df.withColumnRenamed('ENCOUNTER','DIM_ProcedureId')

display(pro_cleaned)

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, caf1dbb6-721f-4e29-8fc3-eed36915afe0)

In [6]:
pro_cleaned.printSchema()

StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 8, Finished, Available, Finished)

root
 |-- START: string (nullable = true)
 |-- STOP: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- DIM_ProcedureId: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_COST: string (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_timestamp

# Correct pattern for timestamps like: 2011-11-30T13:26:36Z
pro_clean = pro_cleaned \
    .withColumn("START", to_timestamp("START", "yyyy-MM-dd'T'HH:mm:ssX")) \
    .withColumn("STOP", to_timestamp("STOP", "yyyy-MM-dd'T'HH:mm:ssX"))

pro_clean.printSchema()

display(pro_clean)


StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 9, Finished, Available, Finished)

root
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- DIM_ProcedureId: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_COST: string (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



SynapseWidget(Synapse.DataFrame, bbd6f7ab-f263-413a-bea5-dd262f163c75)

In [8]:
pro_clean.write.format('delta').mode('overwrite').option("overwriteSchema", "true").save(f"{lakehouse_silver_table}/DIM_Procedures")



StatementMeta(, 2096d9af-b8eb-4ff3-af85-732bcb0b23d8, 10, Finished, Available, Finished)